### Loading Data

### Removing '\n' from the end of every sentence and convert the sentence into lowercase   


### Removing stop words


### Creating bigrams


### Getting a list of unique words

### Creating dictionary of words

### Performing one-hot encoding

### Model